Recommender Sysyem
==

https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101

In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [41]:
from nltk.corpus import stopwords


Import Data
---

In [2]:
#import food rating data
df_food = pd.read_csv('data\\food_rating.csv')
df_food.head()

,타임스탬프,이름 ex) 홍길동,성별,만 나이 ex) 24,찜 (갈비찜/찜닭 등),갈비탕/설렁탕,곱창/막창,볶음밥,김치찌개,된장찌개,...,오므라이스,컵밥,브리또&타코,햄버거,샌드위치,치킨,혼밥은 주로 어떻게 하나요?,혼밥할 때 주로 먹는 음식 메뉴는 무엇인가요? 한 가지만 적어주세요.,(선택사항) 데이트를 하는 상황에서는 주로 무슨 음식을 즐겨 먹나요? 한 가지만 적어주세요.,친한 친구들 여럿이서 만나는 자리에서는 무슨 음식을 즐겨 먹나요? 한 가지만 적어 주세요.
0,10-6-2018 14:32:49,이영건,남자,24,2,4,4,3,2,4,...,3,2,4,5,5,4,집에서 음식을 해 먹는다.,볶음밥,파스타,NaN
1,10-6-2018 15:18:04,성창민,남자,20,5,5,0,5,5,5,...,5,4,5,1,1,3,밖에서 사 먹는다.,덥밥,데이트안함,고기꾸어먹음
2,10-6-2018 15:19:56,윤혜진,여자,20,4,4,3,4,3,5,...,4,3,2,4,3,4,밖에서 사 먹는다.,알밥,파스타,떡볶이
3,10-6-2018 15:21:10,한상욱,남자,21,4,4,5,4,4,2,...,4,2,3,3,4,4,밖에서 사 먹는다.,제육덮밥,스테이크,막창구이
4,10-6-2018 15:21:55,황준원,남자,20,3,5,3,5,4,4,...,5,4,5,5,5,5,배달을 시킨다.,햄버거,양식(파스타),양 많은 것(닭갈비)


In [3]:
df_food_reset_index = df_food.reset_index()
df_food_reset_index = df_food_reset_index.rename(columns = {'index':'userid'})
df_food_reset_index.head()

,userid,타임스탬프,이름 ex) 홍길동,성별,만 나이 ex) 24,찜 (갈비찜/찜닭 등),갈비탕/설렁탕,곱창/막창,볶음밥,김치찌개,...,오므라이스,컵밥,브리또&타코,햄버거,샌드위치,치킨,혼밥은 주로 어떻게 하나요?,혼밥할 때 주로 먹는 음식 메뉴는 무엇인가요? 한 가지만 적어주세요.,(선택사항) 데이트를 하는 상황에서는 주로 무슨 음식을 즐겨 먹나요? 한 가지만 적어주세요.,친한 친구들 여럿이서 만나는 자리에서는 무슨 음식을 즐겨 먹나요? 한 가지만 적어 주세요.
0,0,10-6-2018 14:32:49,이영건,남자,24,2,4,4,3,2,...,3,2,4,5,5,4,집에서 음식을 해 먹는다.,볶음밥,파스타,NaN
1,1,10-6-2018 15:18:04,성창민,남자,20,5,5,0,5,5,...,5,4,5,1,1,3,밖에서 사 먹는다.,덥밥,데이트안함,고기꾸어먹음
2,2,10-6-2018 15:19:56,윤혜진,여자,20,4,4,3,4,3,...,4,3,2,4,3,4,밖에서 사 먹는다.,알밥,파스타,떡볶이
3,3,10-6-2018 15:21:10,한상욱,남자,21,4,4,5,4,4,...,4,2,3,3,4,4,밖에서 사 먹는다.,제육덮밥,스테이크,막창구이
4,4,10-6-2018 15:21:55,황준원,남자,20,3,5,3,5,4,...,5,4,5,5,5,5,배달을 시킨다.,햄버거,양식(파스타),양 많은 것(닭갈비)


Divide Data into User information, and Rating
--

In [4]:
user_info_index = [0,1,2,3,4]  + list(range(54,58))
food_drop_index = list(range(0,5)) + list(range(54,58))

In [5]:
df_user_info = df_food_reset_index.iloc[:,user_info_index]
df_food_rating = df_food_reset_index.drop(axis =1,columns=df_food_reset_index.columns[food_drop_index])
df_food_rating.head()

,찜 (갈비찜/찜닭 등),갈비탕/설렁탕,곱창/막창,볶음밥,김치찌개,된장찌개,닭갈비,닭도리탕,불고기,냉면(물/비빔),...,카레/커리,김밥,분식(떡볶이/튀김/순대),라면,오므라이스,컵밥,브리또&타코,햄버거,샌드위치,치킨
0,2,4,4,3,2,4,3,1,3,4,...,3,3,3,4,3,2,4,5,5,4
1,5,5,0,5,5,5,2,2,5,2,...,3,3,1,1,5,4,5,1,1,3
2,4,4,3,4,3,5,3,2,4,3,...,3,2,4,4,4,3,2,4,3,4
3,4,4,5,4,4,2,4,5,4,4,...,3,3,5,3,4,2,3,3,4,4
4,3,5,3,5,4,4,5,3,5,5,...,5,5,4,5,5,4,5,5,5,5


In [6]:
df_user_info.head()

,userid,타임스탬프,이름 ex) 홍길동,성별,만 나이 ex) 24,혼밥은 주로 어떻게 하나요?,혼밥할 때 주로 먹는 음식 메뉴는 무엇인가요? 한 가지만 적어주세요.,(선택사항) 데이트를 하는 상황에서는 주로 무슨 음식을 즐겨 먹나요? 한 가지만 적어주세요.,친한 친구들 여럿이서 만나는 자리에서는 무슨 음식을 즐겨 먹나요? 한 가지만 적어 주세요.
0,0,10-6-2018 14:32:49,이영건,남자,24,집에서 음식을 해 먹는다.,볶음밥,파스타,NaN
1,1,10-6-2018 15:18:04,성창민,남자,20,밖에서 사 먹는다.,덥밥,데이트안함,고기꾸어먹음
2,2,10-6-2018 15:19:56,윤혜진,여자,20,밖에서 사 먹는다.,알밥,파스타,떡볶이
3,3,10-6-2018 15:21:10,한상욱,남자,21,밖에서 사 먹는다.,제육덮밥,스테이크,막창구이
4,4,10-6-2018 15:21:55,황준원,남자,20,배달을 시킨다.,햄버거,양식(파스타),양 많은 것(닭갈비)


In [7]:
df_food_rating_stack = pd.DataFrame(df_food_rating.stack()).reset_index() 
df_food_rating_stack = shuffle(df_food_rating_stack)
df_food_rating_stack.columns =['personId','contentId','eventStrength']

추천 시스템을 위한 변수 새롭게 설정
--

In [8]:
#articles_df 
#ContentId : 음식 고유번호 
#FoodName : 고유번호에 따른 음식 이름

articles_df = pd.DataFrame(df_food_rating_stack.contentId.unique()).reset_index()
articles_df.columns = ['contentId','foodName'] 

In [9]:
#interactions_full_df 
#personId : 개인 고유번호
#ContentId : 음식 고유번호 
#eventStrength : 음식에 대한 평가

interactions_full_with_zeros_df = df_food_rating_stack.copy().reset_index(drop=True)

for food in range(len(articles_df.foodName)):
    interactions_full_with_zeros_df.loc[interactions_full_with_zeros_df.contentId == articles_df.foodName[food],'contentId'] = articles_df.contentId[food]

In [12]:
interactions_df  = interactions_full_with_zeros_df[interactions_full_with_zeros_df['eventStrength'] != 0]

User with at least 5 interaction
--
Recommender systems have a problem known as user cold-start, in which is hard do provide personalized recommendations for users with none or a very few number of consumed items, due to the lack of information to model their preferences.
For this reason, we are keeping in the dataset only users with at leas 5 interactions.

In [13]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 159
# users with at least 5 interactions: 159


In [14]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 7693
# of interactions from users with at least 5 interactions: 7693


users are allowed to rate an food many times, and interact with them in different ratings. Thus, to model the user interest on a given food, we aggregate all the ratings the user has performed in an item by a weighted sum of rating score strength and apply a log transformation to smooth the distribution.

In [15]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 7693


,personId,contentId,eventStrength
0,0,0,2.321928
1,0,1,2.000000
2,0,2,2.321928
3,0,3,2.000000
4,0,4,2.321928
5,0,5,2.000000
6,0,6,1.584963
7,0,7,2.584963
8,0,8,2.321928
9,0,9,2.584963


Model Evaluation
--

Evaluation is important for machine learning projects, because it allows to compare objectivelly different algorithms and hyperparameter choices for models.
One key aspect of evaluation is to ensure that the trained model generalizes for data it was not trained on, using Cross-validation techniques. We are using here a simple cross-validation approach named holdout, in which a random data sample (20% in this case) are kept aside in the training process, and exclusively used for evaluation. All evaluation metrics reported here are computed using the test set.

Ps. **A more robust evaluation approach** could be to split train and test sets by a **reference date**, where **the train set** is composed by **all interactions before that date**, and **the test set** are interactions **after that date**. For the sake of simplicity, we chose the first random approach for this notebook, but you may want to try the second approach to better simulate how the recsys would perform in production predicting "future" users interactions.

In [32]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 6154
# interactions on Test set: 1539


In [33]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [34]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [36]:
#Top-N accuracy metrics consts

#EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 0

class ModelEvaluator:
    
    def set_random_sample_non_interacted(self,EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS):
        self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.core.series.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32)) ## seed=item_id%(2**32)
            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()   

Popularity Based Recommneder
==

In [87]:
df_food_grouped = interactions_full_df.groupby(['contentId']).agg({'eventStrength': 'sum'}).reset_index()
grouped_rating = interactions_full_df.groupby(['contentId']).agg({'personId': 'count'}).reset_index()
df_food_grouped['eventStrength']  = df_food_grouped['eventStrength'].div(grouped_rating['personId'])
df_food_ranking = df_food_grouped.sort_values(['eventStrength', 'contentId'], ascending = [0,1])

df_food_ranking['foodName'] = ''

for food in range(len(articles_df.contentId)):
    df_food_ranking.loc[df_food_ranking.contentId == articles_df.contentId[food],'foodName'] = articles_df.foodName[food]

df_food_ranking.index = range(1,len(df_food_ranking)+1)

Most Favored 10 Foods
--

In [88]:
df_food_ranking.head(10)

,contentId,eventStrength,foodName
1,2,2.401470,삼겹살(구이)
2,14,2.396908,초밥
3,9,2.393494,스테이크
4,41,2.378197,치킨
5,7,2.344726,파스타
6,26,2.344210,수육/보쌈
7,42,2.316628,회(사시미)
8,35,2.298451,피자
9,5,2.285019,분식(떡볶이/튀김/순대)
10,3,2.284148,닭갈비


Least Favored 10 Foods
--

In [89]:
df_food_ranking.tail(10)

,contentId,eventStrength,foodName
40,23,2.113050,삼계탕
41,30,2.108990,김밥
42,11,2.099598,베트남 쌀국수
43,40,2.082919,샌드위치
44,10,2.069607,비빔밥
45,48,2.056446,수제비
46,31,2.036338,함박스테이크
47,32,2.032874,월남쌈
48,15,1.968258,컵밥
49,21,1.806685,콩국수


Food Recommender
==

Collaborative Filtering Model
==

In [58]:
users_items_pivot_matrix_df = interactions_train_df.pivot(index='personId', 
                                                          columns='contentId', 
                                                          values='eventStrength').fillna(0)

users_items_pivot_matrix_df.head(10)

contentId,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
personId,,,,,,,,,,,,,,,,,,,,,
0,2.321928,0.000000,2.321928,0.000000,2.321928,2.000000,1.584963,2.584963,0.000000,2.584963,...,1.584963,2.584963,2.321928,2.584963,2.321928,2.321928,2.000000,2.321928,2.584963,1.000000
1,0.000000,2.584963,2.584963,1.584963,1.584963,1.000000,0.000000,0.000000,0.000000,0.000000,...,2.584963,1.000000,2.000000,0.000000,2.584963,2.584963,0.000000,2.584963,0.000000,1.584963
2,2.000000,0.000000,2.321928,2.000000,2.000000,2.321928,2.321928,2.321928,2.000000,0.000000,...,0.000000,2.000000,2.321928,2.321928,2.321928,0.000000,2.000000,1.584963,1.584963,2.321928
3,2.321928,0.000000,2.584963,2.321928,2.321928,2.584963,2.000000,2.000000,2.584963,2.584963,...,2.321928,0.000000,2.321928,2.000000,0.000000,2.321928,0.000000,2.000000,0.000000,2.321928
4,2.584963,2.321928,2.584963,2.584963,2.584963,2.321928,0.000000,2.584963,2.000000,2.584963,...,0.000000,0.000000,2.584963,0.000000,2.321928,2.584963,2.584963,2.584963,2.584963,1.584963
5,2.584963,2.584963,2.584963,2.000000,2.321928,2.584963,2.584963,2.584963,2.584963,2.584963,...,0.000000,2.000000,2.584963,2.584963,0.000000,0.000000,2.584963,2.321928,2.584963,2.321928
6,2.000000,2.321928,2.321928,0.000000,2.584963,0.000000,2.584963,2.584963,2.584963,2.000000,...,2.584963,1.584963,2.584963,2.000000,2.000000,0.000000,2.000000,1.584963,2.584963,2.321928
7,0.000000,2.584963,2.321928,2.321928,2.321928,2.321928,0.000000,0.000000,2.321928,2.584963,...,2.321928,2.000000,2.321928,2.321928,2.321928,0.000000,2.321928,2.321928,2.321928,2.321928
8,2.000000,2.584963,2.000000,2.321928,2.000000,2.000000,2.000000,2.000000,1.584963,2.000000,...,2.000000,2.321928,2.000000,0.000000,0.000000,1.584963,1.584963,0.000000,2.321928,2.584963


In [59]:
users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_items_pivot_matrix[:10]

C:\Users\taeny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[2.32192809, 0.        , 2.32192809, 0.        , 2.32192809,
        2.        , 1.5849625 , 2.5849625 , 0.        , 2.5849625 ,
        2.        , 1.5849625 , 2.        , 0.        , 2.5849625 ,
        1.5849625 , 1.5849625 , 0.        , 2.        , 0.        ,
        1.5849625 , 2.        , 2.        , 2.        , 0.        ,
        0.        , 2.5849625 , 0.        , 2.        , 2.5849625 ,
        2.        , 2.        , 1.5849625 , 2.32192809, 2.        ,
        2.5849625 , 2.32192809, 2.        , 0.        , 1.5849625 ,
        2.5849625 , 2.32192809, 2.5849625 , 2.32192809, 2.32192809,
        2.        , 2.32192809, 2.5849625 , 1.        ],
       [0.        , 2.5849625 , 2.5849625 , 1.5849625 , 1.5849625 ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        2.        , 1.        , 2.5849625 , 2.32192809, 2.5849625 ,
        0.        , 1.5849625 , 2.5849625 , 2.5849625 , 2.5849625 ,
        2.5849625 , 2.5849625 , 1.        , 1.        , 0. 

In [60]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [61]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [62]:
users_items_pivot_matrix.shape

(159, 49)

In [63]:
U.shape

(159, 15)

In [64]:
Vt.shape

(15, 49)

In [65]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [66]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings[:1]

array([[1.71278505, 0.89029432, 1.62719796, 0.11297268, 1.87997476,
        1.42837028, 0.92061986, 1.66492732, 0.71423858, 2.02646025,
        1.63948952, 2.51489166, 1.60626399, 0.78593754, 2.20450837,
        1.20042211, 1.5194099 , 1.58486387, 2.66667038, 1.69242986,
        1.06655765, 1.37728413, 1.77151125, 1.75122738, 0.59187756,
        0.90721933, 1.91753257, 0.55286413, 2.54163691, 2.31923914,
        2.39029692, 2.41246641, 1.31148805, 1.76003766, 2.25690292,
        2.03206728, 1.90548235, 1.98130004, 1.63881269, 1.57042786,
        2.20705288, 2.35908727, 1.81166658, 2.72506003, 1.3696958 ,
        1.9829972 , 2.71152868, 2.00348178, 1.17954082]])

In [67]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,149,150,151,152,153,154,155,156,157,158
contentId,,,,,,,,,,,,,,,,,,,,,
0,1.712785,1.398083,2.284991,2.172950,2.544608,1.368194,0.934031,1.832365,2.110437,0.647653,...,1.684746,2.797034,2.756249,2.332335,1.716139,1.789341,2.067929,2.046621,2.707254,0.863511
1,0.890294,1.133530,0.444556,0.364280,2.166272,2.937892,3.332247,2.432721,2.275893,2.773903,...,1.314211,-0.612730,1.577546,1.778007,3.011339,2.124073,2.305827,-0.393542,0.073718,1.421860
2,1.627198,1.973664,1.901933,2.531187,2.668616,2.388614,1.781872,1.510936,1.423036,2.592717,...,3.165913,3.326550,2.598916,2.351479,2.212108,1.697066,1.878050,2.837891,2.726821,2.250485
3,0.112973,0.943066,1.735431,3.249276,1.666830,1.757141,1.089418,1.343829,1.658782,1.127365,...,1.739609,2.538854,2.103407,3.182434,1.062939,1.235197,2.130551,0.630399,2.994895,1.892913
4,1.879975,1.665425,2.251458,1.966004,2.043413,2.382797,2.491163,2.950095,2.494163,1.414634,...,1.547439,2.352986,1.506093,0.845639,2.106590,2.806503,1.128710,1.878221,0.565970,1.997011
5,1.428370,1.610355,2.529484,3.040934,1.766373,1.910614,0.984853,1.653656,1.866834,2.007285,...,2.260116,2.454966,2.547521,1.819091,2.637618,2.678538,2.584460,2.338833,1.205953,2.150633
6,0.920620,1.296483,0.876305,1.547085,0.966210,1.891485,2.824230,1.605586,1.812577,2.420499,...,2.087861,1.196360,2.040787,3.097755,2.186531,1.576202,1.738382,0.378993,2.534094,1.218093
7,1.664927,1.406662,1.577743,2.769432,2.932131,1.273661,2.132073,1.566740,2.105598,2.588893,...,0.953214,2.462776,2.073189,2.179479,1.653774,2.475473,2.382678,2.500329,1.520328,1.678636
8,0.714239,0.662266,1.971855,2.078031,2.442406,3.348528,2.040545,1.997858,2.452592,1.535313,...,2.636056,2.385948,1.920227,2.583439,1.995081,2.197966,0.890579,2.169841,1.803115,1.559453


In [68]:
item_ids = articles_df['contentId'].tolist()

In [69]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = interactions_train_df.pivot(index='personId', 
                                                          columns='contentId', 
                                                          values='eventStrength').fillna(0)

users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_ids = list(users_items_pivot_matrix_df.index)
 
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

sigma = np.diag(sigma)

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 

#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()

C:\Users\taeny\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [82]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['contentId'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'foodName']]


        return recommendations_df
    
cf_recommender_model = CFRecommender(cf_preds_df, articles_df)

In [91]:
personNum = int(input('PersonId를 입력해주세요 : '))

cf_model = cf_recommender_model.recommend_items(personNum,verbose=True)

for food in range(len(articles_df.contentId)):
    cf_model.loc[cf_model.contentId == articles_df.contentId[food],'foodName'] = articles_df.foodName[food]

cf_model.index = range(1,len(cf_model)+1)

cf_model

PersonId를 입력해주세요 :  10


,recStrength,contentId,foodName
1,3.170933,47,리조또
2,2.779551,31,함박스테이크
3,2.712729,14,초밥
4,2.609888,16,짬뽕
5,2.543780,1,게장
6,2.502029,43,냉모밀
7,2.436255,23,삼계탕
8,2.310504,2,삼겹살(구이)
9,2.297931,20,김치찌개
10,2.297430,4,냉면(물/비빔)


In [92]:
personNum = int(input('PersonId를 입력해주세요 : '))

cf_model = cf_recommender_model.recommend_items(personNum,verbose=True)

for food in range(len(articles_df.contentId)):
    cf_model.loc[cf_model.contentId == articles_df.contentId[food],'foodName'] = articles_df.foodName[food]

cf_model.index = range(1,len(cf_model)+1)

cf_model

PersonId를 입력해주세요 :  1


,recStrength,contentId,foodName
1,2.844927,28,오므라이스
2,2.394232,21,콩국수
3,2.381648,14,초밥
4,2.285118,11,베트남 쌀국수
5,2.226920,43,냉모밀
6,2.192646,34,카레/커리
7,2.099007,44,갈비탕/설렁탕
8,2.024204,26,수육/보쌈
9,1.973664,2,삼겹살(구이)
10,1.926572,23,삼계탕


In [190]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
158 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 1.0, 'recall@10': 1.0}


,_person_id,hits@10_count,hits@5_count,interacted_count,recall@10,recall@5
0,119,10,10,10,1.0,1.0
108,58,10,10,10,1.0,1.0
104,21,10,10,10,1.0,1.0
102,140,10,10,10,1.0,1.0
101,20,10,10,10,1.0,1.0
100,100,10,10,10,1.0,1.0
99,16,10,10,10,1.0,1.0
98,66,10,10,10,1.0,1.0
97,64,10,10,10,1.0,1.0
95,86,10,10,10,1.0,1.0


Content-based Filtering
==

In [45]:
stopwords_list = []
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

item_ids = articles_df['contentId'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_df['foodName'])
#foodName, foodDescribtion!!!
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

<49x81 sparse matrix of type '<class 'numpy.float64'>'
	with 81 stored elements in Compressed Sparse Row format>

In [47]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_full_df[interactions_full_df['contentId'] \
                                                   .isin(articles_df['contentId'])].set_index('personId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [48]:
user_profiles = build_users_profiles()
len(user_profiles)

159

In [51]:
myprofile = user_profiles[1]
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[1].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

(1, 81)


,token,relevance
0,게장,0.189435
1,김치찌개,0.189435
2,냉모밀,0.189435
3,된장찌개,0.189435
4,볶음밥,0.189435
5,불고기,0.189435
6,오므라이스,0.189435
7,잔치국수,0.189435
8,제육볶음,0.189435
9,초밥,0.189435


In [84]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'foodName']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(articles_df)

In [85]:
personNum = int(input('PersonId를 입력해주세요 : '))

cb_model = content_based_recommender_model.recommend_items(personNum,verbose=True)

for food in range(len(articles_df.contentId)):
    cb_model.loc[cb_model.contentId == articles_df.contentId[food],'foodName'] = articles_df.foodName[food]

cb_model.index = range(1,len(cb_model)+1)

cb_model

PersonId를 입력해주세요 :  1


,recStrength,contentId,foodName
1,0.189435,18,돈부리(일본식 덮밥)
2,0.189435,0,"해장국(뼈해장국, 순대국 등)"
3,0.189435,1,게장
4,0.189435,43,냉모밀
5,0.189435,21,콩국수
6,0.189435,20,김치찌개
7,0.189435,33,된장찌개
8,0.189435,12,볶음밥
9,0.189435,14,초밥
10,0.189435,28,오므라이스


In [56]:
model_evaluator.set_random_sample_non_interacted(0)
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
158 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 1.0, 'recall@10': 1.0}


,_person_id,hits@10_count,hits@5_count,interacted_count,recall@10,recall@5
0,119,10,10,10,1.0,1.0
108,58,10,10,10,1.0,1.0
104,21,10,10,10,1.0,1.0
102,140,10,10,10,1.0,1.0
101,20,10,10,10,1.0,1.0
100,100,10,10,10,1.0,1.0
99,16,10,10,10,1.0,1.0
98,66,10,10,10,1.0,1.0
97,64,10,10,10,1.0,1.0
95,86,10,10,10,1.0,1.0
